<h1>"Викишоп". Анализ токсичности комментариев с помощью BERT.<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Цель-и-задачи-проекта." data-toc-modified-id="Цель-и-задачи-проекта.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Цель и задачи проекта.</a></span></li><li><span><a href="#Входные-данные." data-toc-modified-id="Входные-данные.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Входные данные.</a></span><ul class="toc-item"><li><span><a href="#Библиотеки." data-toc-modified-id="Библиотеки.-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Библиотеки.</a></span></li><li><span><a href="#Загрузка-и-обзор." data-toc-modified-id="Загрузка-и-обзор.-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Загрузка и обзор.</a></span></li></ul></li><li><span><a href="#Предобработка-текста." data-toc-modified-id="Предобработка-текста.-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Предобработка текста.</a></span><ul class="toc-item"><li><span><a href="#Токенизация." data-toc-modified-id="Токенизация.-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Токенизация.</a></span></li><li><span><a href="#Выравнивание-длин-текстов." data-toc-modified-id="Выравнивание-длин-текстов.-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Выравнивание длин текстов.</a></span></li><li><span><a href="#Маска-&quot;внимания&quot;." data-toc-modified-id="Маска-&quot;внимания&quot;.-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Маска "внимания".</a></span></li></ul></li><li><span><a href="#Создание-признаков-эмбеддингов-с-BERT." data-toc-modified-id="Создание-признаков-эмбеддингов-с-BERT.-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Создание признаков-эмбеддингов с BERT.</a></span></li><li><span><a href="#Обучение-моделей.-Выбор-лучшей." data-toc-modified-id="Обучение-моделей.-Выбор-лучшей.-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Обучение моделей. Выбор лучшей.</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия." data-toc-modified-id="Логистическая-регрессия.-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Логистическая регрессия.</a></span></li><li><span><a href="#Случайный-лес." data-toc-modified-id="Случайный-лес.-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Случайный лес.</a></span></li><li><span><a href="#Дамми-модель." data-toc-modified-id="Дамми-модель.-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Дамми-модель.</a></span></li><li><span><a href="#Выводы." data-toc-modified-id="Выводы.-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Выводы.</a></span></li></ul></li></ul></div>

# Цель и задачи проекта.

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

`Дано:`

Массив данных, включающий в себя 159571 комментарий пользователей, которые они оставляли к описаниям товаров на сайте "Викишоп". Для каждого комментария выставлен индикатор токсичности:  0 - комментарий нетоксичный, 1 - токсичный.

`Цель:`

Разработать инструмент - модель машинного обучения - который сможет определять токсичность новых комментариев, то есть классифицировать их на позитивные и негативные.

`Задачи:`

1. Посмотреть на датасет. Проверить на пропуски/повторы.
2. Подготовить признаки для машинного обучения силами BERT.
3. Обучить логистическую регрессию и случайный лес. Найти лучшую модель с лучшими гиперпараметрами и предложить её для классификации комментариев.


# Входные данные.

## Библиотеки.

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
import torch
import transformers
import warnings
warnings.filterwarnings('ignore')
from tqdm import notebook

## Загрузка и обзор.

Загрузим наш датасет. Здесь и далее в адресной строке указан локальный адрес расположения файлов.

In [2]:
try:

    df = pd.read_csv("/Users/mihasia/Documents/Yandex_Practikum_Data_Science/Sprint_13(12)/toxic_comments.csv")

except:

    df = pd.read_csv("/datasets/toxic_comments.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Много-много буков. Если мы собираемся использовать BERT, то такое количество текстов нам не осилить.

Пропусков не наблюдается, что славно.

In [4]:
df.duplicated().sum()

0

Дубликатов тоже нет. Посмотрим на начинку датасета.

In [5]:
pd.set_option('display.max_colwidth', -1)
df.sample(10)

,text,toxic
27058,"Speedy Deletion \n\nPlease do not make personal attacks. Wikipedia has a strict policy against personal attacks. Attack pages and images are not tolerated by Wikipedia and are speedily deleted. Users who continue to create or repost such pages and images, especially those in violation of our Wikipedia:Biographies of living persons policy, will be blocked from editing Wikipedia. Thank you.",0
150995,"""\n\n Accolades \n\nRather than get into an edit-war, I'll post my opinion here about the so-called """"accolades"""". What we are talking about here is a nomination-for-a-nomination, not a nomination, and certainly not a win. How is this notable? -''''''The '45 """,0
11302,"""\n\nFro anyone who cared to pay attention, it was obvious that """"eight and aces"""" was a typo. """"Eights and Aces"""" is correct. """"Aces and Eights"""" is also valid. ]] 00:12, 17 Sep 2004 (UTC)\n\n""",0
8030,I think this article is a little biased.... I don't particularly recall the old format being hugely popular. I do know that I love 88.3's current format. I can almost always turn it on and find a song I know.,0
72875,Anthem \n\nDid they play La Marsalles (sorry for the poor spelling) alongside God Save The Queen before the match due to the fact that the Dragons were French,0
54481,}}}}}\n\n{{PageTabs\n |Talk page/\n |Sanctions I am aware of Please place any GS or DS notices on this page\n \n |This={{{This|1,0
40548,"Definitely this page is not written from a neutral point of view and has no citations whatsoever, the article needs a change of focus an citation from different sources.\n\n''''''(talk)",0
4191,"""\n\n A beer for you! \n\n good work your doing well """,0
98554,IronRuby \nJust wanted to note that IronRuby is not an web application framework like you added. This really might confuse people. I'll fix that one.,0
106645,"""\nThanks ) Was rather interesting being in the news... Zhang Help resolve disputes! """,0


Можно "залипнуть" на комментариях, помеченных единицей, поистине, ругательства и обзывательства порой впечатляют побольше заурядной похвалы. И нам предстоит научить компьютер их различать самостоятельно. 

# Предобработка текста.

Как уже отмечалось выше, объём датасета "неподъёмный" для системы BERT, тем более на локальном слабом ноутбуке. Скрепя сердце вырежем из датасета 2000 рандомных объектов.

In [6]:
# машина не потянет весь датасет, возьмём 2000 рандомных строк
df = df.sample(2000)

При использовании BERT нам нужно сделать следующие предобработки:
- токенизация
- выравнивание длин текстов
- создание маски "внимания" для распознавания нужных токенов

## Токенизация.

Итак, чтобы превратить тексты в токены, загрузим словарик из базы данных модели BERT. Мы будем использовать самый простой вариант, чтобы не перегружать память - "bert-base-uncased". Словарь был скачан [вот отсюда](https://huggingface.co/bert-base-uncased/tree/main) (файл `vocab.txt`) и загружен в проект по локальному пути.  

Инициализируем токенизатор.

In [7]:
try:
    
    tokenizer = transformers.BertTokenizer("/Users/mihasia/Documents/Yandex_Practikum_Data_Science/Sprint_13(12)/vocab2.txt")

except:
    
    tokenizer = transformers.BertTokenizer("https://huggingface.co/bert-base-uncased/blob/main/vocab.txt")

Теперь сделаем преобразование текстов в номера токенов из словаря, загруженного выше. В процессе работы также было отмечено, что тексты превышают предельно допустимую в этом методе длину в 512 токенов, поэтому вынужденно устанавливаем ограничение, остальное при этом обрежется. Будем надеяться, что для обучения это не фатально, и моделям хватит и того, что есть.

In [8]:
# токенизируем наш срез датасета, обрезая слишком длинные тексты, чтобы пройти ограничение в 512
tokenized = df["text"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512)))

## Выравнивание длин текстов.

Как было уже отмечено, тексты попадаются разной длины, что неудивительно, как и наличие "болтунов" и людей "меньше слов, больше дела!". Исправим это недоразумение, тем более, что BERT обидится, если ему преподнести данные как-то иначе. 

Мы искусственно удлинним каждый текст до максимума (в нашем случае - 512), и в образовавшиеся ячейки-пустоты проставим нули.

In [9]:
# сделаем длины токенизированных текстов одинаковыми и заполним нулями не занятые "хвосты"
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
padded.shape

(2000, 512)

Отлично, размер датасета совпадает с ожидаемым. Идём дальше.

## Маска "внимания".

Маска внимания нужна, чтобы прикрыть то нулевое безобразие, что мы насоздавали в прошлом разделе проекта. Нужно, чтобы BERT исследовал только существенные токены, то есть те, которые были нашими изначальными текстами - вот и создадим для него матрицу из нулей и единиц такого же, как датасет, размера. Единицы в нём будут обозначать искомые токены.

In [10]:
# создадим маску-шпаргалку для модели, чтобы ей было понятно: на проставленных выше нулях учиться не надо
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 512)

Убедились, размер тот же. Вперёд!

# Создание признаков-эмбеддингов с BERT.

Начнём с инициализации конфигурации и модели. Как и словарь токенов, эти файлы были получены [вот отсюда](https://huggingface.co/bert-base-uncased/tree/main) (файлы `config.json` и `pytorch_model.bin`, соответственно), и запущены в проект с локальной машины.

При инициализации модели не забудем указать параметр конфигурации.

In [11]:
try:
    
    config = transformers.BertConfig.from_json_file(
    "/Users/mihasia/Documents/Yandex_Practikum_Data_Science/Sprint_13(12)/config2.json")
    
except:
    
    config = transformers.BertConfig.from_json_file(
    "https://huggingface.co/bert-base-uncased/blob/main/config.json")

In [12]:
try:
    
    model = transformers.BertModel.from_pretrained(
    "/Users/mihasia/Documents/Yandex_Practikum_Data_Science/Sprint_13(12)/pytorch_model2.bin",
    config=config)
    
except:
    
    model = transformers.BertModel.from_pretrained(
    "https://huggingface.co/bert-base-uncased/blob/main/pytorch_model.bin",   
    config=config)

Some weights of the model checkpoint at /Users/mihasia/Documents/Yandex_Practikum_Data_Science/Sprint_13(12)/pytorch_model2.bin were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Предупреждение, которое могло появиться после загрузки, гласит, что некоторые весы не были использованы при инициализации модели, и это ожидаемо, если мы используем BertForPreTraining модель. Мы её и используем, полагаю, всё в порядке.

Приступаем к самой "мясистой" части работы. Нам предстоит запустить долгий цикл "эмбеддингизации" текстов, где за каждый цикл модель будет работать с батчем размером в 200 объектов. Затем мы слепим воедино полученные батчи с эмбеддингами, то есть векторными представлениями текстов - это и будут наши признаки для предсказаний токсичности.

In [13]:
# задаём размер батча
batch_size = 200
# создаём пустой лист для коллекционирования эмбеддингов
embeddings = []
# запускаем цикл батчами со счётчиком времени
for i in notebook.tqdm(range(padded.shape[0]//batch_size)):
    # делаем из данных тензоры (многомерные векторы)
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])  
    # указываем источник маски "внимания"
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
    # поясняем, что не нужен градиентный спуск, потому что будем обучаться потом отдельно
    with torch.no_grad():
        # собираем только "нужные", согласно маске, эмбеддинги из тензора
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    # прилепляем эмбеддинги к общему списку
    embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
features = np.concatenate(embeddings)
#pd.DataFrame(features).to_csv("/Users/mihasia/Documents/Yandex_Practikum_Data_Science/Sprint_13(12)/features.csv")

На локальной машине цикл считался полтора часа, это приличное время, но уменьшать датасет ещё сильнее не хотелось. Уменьшение размеров и увеличение количества батчей давало результат хуже описанного. Для страховки признаки были экспортированы, если вдруг не будет хватать времени на перезапуск всего кода.

In [15]:
#test = pd.read_csv("/Users/mihasia/Documents/Yandex_Practikum_Data_Science/Sprint_13(12)/features.csv")
pd.DataFrame(features).head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.182341,0.206809,-0.362974,0.451316,0.213900,0.102443,0.035701,0.557241,-0.203374,0.287266,...,-0.182744,0.162574,0.410327,-0.116449,0.294475,0.230649,-0.293169,-0.068288,0.685881,0.587387
1,-0.474841,-0.030300,0.352599,-0.094763,-0.142603,-0.632908,-0.086629,0.230572,-0.204094,-0.689256,...,-0.005922,-0.308224,0.128605,-0.420191,-0.044927,0.531814,-0.044234,-0.702617,0.083213,0.165290
2,0.227460,-0.082311,-0.335135,0.084723,-0.679311,-0.643975,0.523805,0.801702,0.201131,-0.359601,...,0.109084,-0.198109,0.169481,-0.310461,0.160826,0.035175,-0.128213,-0.219390,0.371376,0.692709
3,0.218693,-0.194800,-0.245334,0.197041,-0.056888,-0.514260,0.839216,0.281435,0.240447,-0.318882,...,0.305383,-0.000331,0.039010,-0.045343,0.333013,0.180759,-0.026365,-0.201529,0.777940,0.327623
4,-0.144419,0.226945,-0.255751,0.054216,-0.315064,-0.611658,0.632851,0.410951,-0.197194,-0.620301,...,0.707762,-0.256246,0.075699,-0.317784,0.699774,0.006987,-0.619489,-0.092781,0.800038,0.153542


 Признаки готовы. Целевой признак был готовенький сразу. Пошли учиться.

# Обучение моделей. Выбор лучшей.

Традиционное действо перед обучением - разделение выборки на обучающую и тестовую. Поделим 3 к 1.

In [16]:
X = features
y = df.toxic.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## Логистическая регрессия.

Попробуем логистическую регрессию для наших признаков. Чтобы как-то ей помочь, сделаем перебор одного из параметров - силы регуляризации.

In [17]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(X_train, y_train)

print('best parameters: ', grid_search.best_params_)
print('best scores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scores:  0.932


Отлично, тогда зададим этот параметр и обучим модель.

In [18]:
LR = LogisticRegression(C=5.26)
LR.fit(X_train, y_train)

LogisticRegression(C=5.26)

Теперь проверим, насколько качественно предсказываются ответы для обучающей и тестовой выборок, посчитав метрику accuracy.

In [19]:
print("Accuracy for train:", LR.score(X_train, y_train))
print("Accuracy for test:", LR.score(X_test, y_test))

Accuracy for train: 1.0
Accuracy for test: 0.928


Неплохо!

## Случайный лес.

Метрики и так вполне хороши, но почему бы не попробовать ещё одну модель. Подберём глубину и количество деревьев и посмотрим, какая точность классификации получится здесь.

In [20]:
parameters = {'max_depth': np.linspace(1, 21, 2), 'n_estimators': np.linspace(1, 101, 10).astype(int)}
grid_search = GridSearchCV(RandomForestClassifier(), parameters)
grid_search.fit(X_train, y_train)

print('best parameters: ', grid_search.best_params_)
print('best scores: ', grid_search.best_score_)

best parameters:  {'max_depth': 21.0, 'n_estimators': 45}
best scores:  0.9179999999999999


In [21]:
RFC = RandomForestClassifier(max_depth=21, n_estimators=34, random_state=42)
RFC.fit(X_train, y_train)

RandomForestClassifier(max_depth=21, n_estimators=34, random_state=42)

In [22]:
print("Accuracy for train:", RFC.score(X_train, y_train))
print("Accuracy for test:", RFC.score(X_test, y_test))

Accuracy for train: 1.0
Accuracy for test: 0.908


Слегка похуже, чем у логистической регрессии.

## Дамми-модель.

Напоследок убедимся, что классификаторы наши вообще-то вполне адекватны. Запустим дамми-модель на наших данных и посмотрим на score.

In [23]:
DC = DummyClassifier()
DC.fit(X_train, y_train)

print("Accuracy for train:", DC.score(X_train, y_train))
print("Accuracy for test:", DC.score(X_test, y_test))

Accuracy for train: 0.904
Accuracy for test: 0.892


Здесь всё выглядит не очень даже для обучающей выборки.

## Выводы.

1. BERT очень долгий, но он того стоит. Он сам "подчистил" тексты и сам сделал адекватные для обучения признаки.
2. Протестирована всего пара простых моделей для классификации, но среди них уже есть отличные варианты.
3. Лучший вариант - логистическая регрессия с "подкрученным" параметром регуляризации.
4. Ругаться в интернете - нехорошо. Приходится тратить время на такие проекты, чтобы эти ругательства потом подчищать. 